In [3]:
import numpy as np
import scipy
import matplotlib.pyplot as plt 
from numpy.linalg import eig
import matplotlib.animation as animation
from IPython.display import clear_output
import subprocess
import random
from datetime import datetime
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)


Device: cpu


In [4]:
def read_data(file_path):
    with open(file_path, 'r') as file:
        data = file.readlines()

    # Extracting layers
    layers = []
    for line in data:
        if line.strip() == 'Layers:':
            break
    for line in data[data.index('Layers:\n') + 1:]:
        if line.strip() == 'Materials:':
            break
        layer_info = line.strip().replace("(", "").replace(")", "").replace("'", "").split(', ')
        layer = (layer_info[0], float(layer_info[1]))
        layers.append(layer)

    # Extracting materials
    materials = {}
    for line in data[data.index('Materials:\n') + 1:]:
        if line.strip() == 'QWI Target Shift:':
            break
        material_info = line.strip().split(': ')
        material_name = material_info[0]
        properties = [float(prop) for prop in material_info[1][1:-1].split(', ')]
        materials[material_name] = properties

    # Extracting QWI target shift
    qwi_target_shift = float(data[data.index('QWI Target Shift:\n') + 1])

    # Extracting number of electric fields
    number_of_electric_fields = int(data[data.index('Number of Electric Fields:\n') + 1])

    # Extracting max applied electric field
    max_applied_electric_field = int(data[data.index('Max Applied Electric Field:\n') + 1])

    # Extracting FOM elements
    fom_elements = [float(data[data.index('FOM:\n') + i]) for i in range(1, 5)]

    return materials, layers, qwi_target_shift, max_applied_electric_field, fom_elements


# Example usage:
# for files in file_path:
#     file_path = 'data.txt'  # Replace 'your_file_path_here.txt' with the actual file path
#     materials, layers, qwi_target_shift, max_applied_electric_field, fom_elements = read_data(file_path)
#     input_ = [materials, layers, qwi_target_shift, max_applied_electric_field, fom_elements]
#     print(input_)
#     print(fom_elements)

In [5]:
import os
input_features_matrix=[]
figure_of_merit=[]
# Directory path
directory = 'Z:\FYP\combined_data_sets'

# Iterate over all files in the directory
for filename in os.listdir(directory):
    # Check if the file is a text file
    if filename.endswith('.txt'):
        # Open the file
        file_path = os.path.join(directory, filename)
        materials, layers, qwi_target_shift, max_applied_electric_field, fom_elements = read_data(file_path)
        input_ = [materials, layers, qwi_target_shift, max_applied_electric_field, fom_elements]
        input_features_matrix.append(input_)
        

In [6]:
input_features_matrix[1]

[{'GaAs': [0.111, 1.42, 0.067, 0.08, 0.5, 3.9476],
  'GaP': [-0.388, 2.74, 0.25, 0.14, 0.67, 3.3798],
  'InP': [0.0, 1.35, 0.077, 0.12, 0.6, 3.3688],
  'InAs': [0.441, 0.354, 0.023, 0.025, 0.4, 3.714],
  'AlAs': [-0.4245, 2.95, 0.15, 0.16, 0.79, 2.994],
  'In0.528378Ga0.254615Al0.217007As': [0.2232880459876067,
   1.0273682873288552,
   0.061762949,
   0.06829977000000001,
   0.51009423,
   3.52027361518913],
  'In0.5311183Ga0.417546Al0.0513357As': [0.29922348681464206,
   0.8065615387768478,
   0.0478916579,
   0.054895349499999996,
   0.46177552299999997,
   3.539881800967616],
  'In0.5319670000000012Ga0.468033Al-1.11468e-15As': [0.3187091531757405,
   0.7499006886428017,
   0.04359345199999986,
   0.05074181499999985,
   0.44680329999999957,
   3.542847366859467],
  'In0.5295380000000001Ga0.323578Al0.146884As': [0.2578501073178942,
   0.9268679636001914,
   0.0558917,
   0.06262613,
   0.48964256000000006,
   3.529958110383502],
  'In0.52948Ga0.320116Al0.150404As': [0.25619971405192

In [7]:
import numpy as np
from sklearn.model_selection import train_test_split

# Initialize lists to store features and targets
X_train_list = []
y_train_list = []

# Iterate over each heterostructure
for heterostructure in input_features_matrix:
    # Extract features and targets from the current heterostructure
    materials = heterostructure[0]
    layers = heterostructure[1]
    qwi_target_shift = heterostructure[2]
    max_applied_electric_field = heterostructure[3]
    fom_elements = heterostructure[4]

    # Initialize list to store material features in the correct order
    material_features = []

    # Iterate over each layer to ensure correct material ordering
    for layer in layers:
        material_key = layer[0]
        if material_key in materials:
            material_features.extend(materials[material_key])

    # Flatten the layers list
    layer_thicknesses = [thickness for _, thickness in layers]

    # Combine all features into a single array
    all_features = np.concatenate((material_features, layer_thicknesses, [qwi_target_shift], [max_applied_electric_field]))

    # Append features to the features list
    X_train_list.append(all_features)

    # Append targets to the targets list
    y_train_list.append(fom_elements)
    
print(fom_elements)

# Print the shape of each element in X_train_list
# for i, x in enumerate(X_train_list):
#     print(f"Element {i}: Shape = {x.shape}")

# Convert the lists to numpy arrays
X_train = np.array(X_train_list)
y_train = np.array(y_train_list)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

print("Training data shapes:")
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("\nTesting data shapes:")
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)


[0.32175319210395825, 0.04814051282645505, 1.7640811507802625, 0.09042567680878609]
Training data shapes:
X_train shape: (3918, 37)
y_train shape: (3918, 4)

Testing data shapes:
X_test shape: (980, 37)
y_test shape: (980, 4)


In [8]:
import numpy as np
from sklearn.model_selection import train_test_split

# Get the maximum shape among all elements in X_train_list
max_shape = max(x.shape[0] for x in X_train_list)

# Pad the elements to have the same shape
X_train_padded = np.array([np.pad(x, (0, max_shape - x.shape[0]), mode='constant') for x in X_train_list])

# Convert lists to numpy arrays
y_train = np.array(y_train_list)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_train_padded, y_train, test_size=0.025, random_state=42)

print("Training data shapes:")
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("\nTesting data shapes:")
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)


Training data shapes:
X_train shape: (4775, 37)
y_train shape: (4775, 4)

Testing data shapes:
X_test shape: (123, 37)
y_test shape: (123, 4)


In [14]:
import torch
import torch.nn as nn
import torch.optim as optim

class NeuralNetwork(nn.Module):
    def __init__(self, input_size):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, 256)
        #self.fc2 = nn.Linear(128, 64)
        self.fc2 = nn.Linear(256, 256)
        self.fc3 = nn.Linear(256, 4)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Build neural network model
def build_model(input_size):
    model = NeuralNetwork(input_size)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    return model, criterion, optimizer

# Example usage:
input_size = X_train.shape[1]  # Input size corresponds to the number of input features
model, criterion, optimizer = build_model(input_size)
print(model)


NeuralNetwork(
  (fc1): Linear(in_features=37, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=4, bias=True)
)


In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import mean_squared_error
import numpy as np

# Train the model
def train_model(model, criterion, optimizer, X_train, y_train, epochs=2000):
    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, targets in zip(X_train, y_train):
            inputs = inputs.clone().detach().type(torch.float32)
            targets = targets.clone().detach().type(torch.float32)

            optimizer.zero_grad()

            outputs = model(inputs)
            targets = targets.view(-1)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)

        epoch_loss = running_loss / len(X_train)
        if epoch % 10 == 0:
            print(f'Epoch [{epoch+1}/{epochs}], Loss: {epoch_loss:.4f}')

# Evaluate the model
def evaluate_model(model, X_test, y_test):
    model.eval()
    with torch.no_grad():
        inputs = X_test.clone().detach().type(torch.float32)
        targets = y_test.clone().detach().type(torch.float32)

        outputs = model(inputs)
        mse = mean_squared_error(targets.numpy(), outputs.numpy())
        print(f'Mean Squared Error (MSE) on test data: {mse:.4f}')

# Train and evaluate the model
def train_and_evaluate_model(model, criterion, optimizer, X_train, y_train, X_test, y_test):
    train_model(model, criterion, optimizer, X_train, y_train)
    evaluate_model(model, X_test, y_test)

# Convert input features and target values to numpy arrays
X_train_np = np.array(X_train)
y_train_np = np.array(y_train)
X_test_np = np.array(X_test)
y_test_np = np.array(y_test)

# Convert numpy arrays to PyTorch tensors
X_train_tensor = torch.tensor(X_train_np, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_np, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_np, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test_np, dtype=torch.float32)

# Example usage:
epochs = 2000
batch_size = 256
# Train and evaluate the model
train_and_evaluate_model(model, criterion, optimizer, X_train_tensor, y_train_tensor, X_test_tensor, y_test_tensor)

Epoch [1/2000], Loss: 12.1151
Epoch [11/2000], Loss: 8.6527
Epoch [21/2000], Loss: 8.3563
Epoch [31/2000], Loss: 8.1977
Epoch [41/2000], Loss: 8.0779
Epoch [51/2000], Loss: 7.9622
Epoch [61/2000], Loss: 7.7552
Epoch [71/2000], Loss: 7.4171
Epoch [81/2000], Loss: 7.1147
Epoch [91/2000], Loss: 7.5675
Epoch [101/2000], Loss: 7.4870
Epoch [111/2000], Loss: 7.4449
Epoch [121/2000], Loss: 7.4378
Epoch [131/2000], Loss: 7.3740
Epoch [141/2000], Loss: 7.3210
Epoch [151/2000], Loss: 7.2894
Epoch [161/2000], Loss: 7.2502
Epoch [171/2000], Loss: 7.1975
Epoch [181/2000], Loss: 7.2014
Epoch [191/2000], Loss: 7.1765
Epoch [201/2000], Loss: 7.2114
Epoch [211/2000], Loss: 7.1473
Epoch [221/2000], Loss: 7.0974
Epoch [231/2000], Loss: 7.0720
Epoch [241/2000], Loss: 7.0315
Epoch [251/2000], Loss: 6.9792
Epoch [261/2000], Loss: 6.9726
Epoch [271/2000], Loss: 6.9683
Epoch [281/2000], Loss: 6.9466
Epoch [291/2000], Loss: 7.0160
Epoch [301/2000], Loss: 6.9296
Epoch [311/2000], Loss: 6.9219
Epoch [321/2000], 

In [56]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)


Device: cpu


In [16]:

from datetime import datetime
# from your_module import NeuralNetwork  # Import your neural network class from the module where it's defined

# Save the entire model with timestamp
current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
model_path = f'trained_model_{current_time}.pth'
torch.save(model, model_path)


In [17]:
model_path = r'C:\Users\jmcc0\Desktop\FYP\QWI\trained_model_2024-04-08_00-28-10.pth'
model = torch.load(model_path)

In [18]:
# Assuming you have a trained model named 'model'
file_path = f"Z:\FYP\data_2024-04-07_17-52-46.txt"

# Read the data from the file
materials, layers, qwi_target_shift, max_applied_electric_field, fom_elements = read_data(file_path)

# Extract material names from the 'Layers' data in order of appearance
material_names_ordered = [layer[0] for layer in layers]

# Flatten the materials dictionary in the order of appearance
material_features = []
for material_key in material_names_ordered:
    if material_key in materials:
        material_features.extend(materials[material_key])

# Flatten the layers list
layer_thicknesses = [thickness for _, thickness in layers]

# Combine all features into a single array
all_features = np.concatenate((material_features, layer_thicknesses, [qwi_target_shift], [max_applied_electric_field]))

input_tensor = torch.tensor(all_features, dtype=torch.float32)
print(input_tensor)
# Pass input data through the model
with torch.no_grad():
    model.eval()  # Set the model to evaluation mode
    output_tensor = model(input_tensor)

# Get the predicted values
predicted_values = output_tensor.numpy()

print("Predicted values:", predicted_values)


tensor([1.4834e-01, 1.2453e+00, 7.2861e-02, 7.9024e-02, 5.4875e-01, 3.5187e+00,
        3.1183e-01, 7.6989e-01, 4.5159e-02, 5.2255e-02, 4.5226e-01, 3.5420e+00,
        3.1516e-01, 7.6022e-01, 4.4409e-02, 5.1530e-02, 4.4964e-01, 3.5424e+00,
        1.3747e-01, 1.2769e+00, 7.4332e-02, 8.0445e-02, 5.5388e-01, 3.5246e+00,
        1.4834e-01, 1.2453e+00, 7.2861e-02, 7.9024e-02, 5.4875e-01, 3.5187e+00,
        1.0000e+02, 3.8176e+01, 4.6050e+01, 5.8135e+01, 1.0000e+02, 3.1250e+01,
        1.0000e+01])
Predicted values: [1.2232358  0.17685056 1.3957118  0.1140147 ]


In [26]:
from skopt import gp_minimize
from skopt.space import Real
import numpy as np

num_layers=5
def objective_function(params):
    layer_thicknesses = params[:num_layers]
    bandgaps = params[num_layers:]

    layers = [Layer(InGaAlAs_material(bandgap), thickness) for bandgap, thickness in zip(bandgaps, layer_thicknesses)]
    if check:
        scores = model_evaluation(layers)  # Replace with your model evaluation function
        return sum(scores)  # Return the sum of scores as the objective
    else:
        return 1e6

def constraint_check(params):
    layer_thicknesses = params[:num_layers]
    bandgaps = params[num_layers:]

    layers = [Layer(InGaAlAs_material(bandgap), thickness) for bandgap, thickness in zip(bandgaps, layer_thicknesses)]
    layers[0].thickness = 100
    layers[-1].thickness = 100
    
    if (getGap(layers) > 1499 and getGap(layers) < 1551): 
        check = True
    else: 
        check = False
    clear_output()
    print(check)
    return check

def objective_with_constraint(params):
    if constraint_check(params):
        return objective_function(params)
    else:
        return 1e6

# Define the search space
search_space = []
for _ in range(num_layers):
    search_space.append(Real(30, 100))  # Adjust the range as needed
for _ in range(num_layers):
    search_space.append(Real(900, 1650))  # Adjust the range as needed

# Perform Bayesian optimization
result = gp_minimize(objective_with_constraint, search_space, n_calls=50)

# Extract the optimized parameters
best_params = result.x
best_score = result.fun

# Output the results
print("Best Parameters:", best_params)
print("Best Score:", best_score)


True
x = 0.253914
y = 0.217719
x=0.253914 y=0.217719
BG:1.0284407073758248
x = 0.308618
y = 0.162095
x=0.308618 y=0.162095
BG:0.9477931015928155
x = 0.379309
y = 0.0902157
x=0.379309 y=0.0902157
BG:0.8531917373254098
x = 0.453459
y = 0.0148187
x=0.453459 y=0.0148187
BG:0.7656815127529981
x = 0.158675
y = 0.31456
x=0.158675 y=0.31456
BG:1.1842281826514602
x = 0.250693
y = 0.220995
x=0.250693 y=0.220995
BG:1.033392844767674
x = 0.305655
y = 0.165109
x=0.305655 y=0.165109
BG:0.9519982016507235
x = 0.376669
y = 0.0929006
x=0.376669 y=0.0929006
BG:0.8565302629403297
x = 0.451452
y = 0.0168596
x=0.451452 y=0.0168596
BG:0.7678917118824079
x = 0.157657
y = 0.315595
x=0.157657 y=0.315595
BG:1.1859980945553752
1501.5142286636644
x = 0.249845
y = 0.221857
x=0.249845 y=0.221857
BG:1.0346991511190275
x = 0.304875
y = 0.165902
x=0.304875 y=0.165902
BG:0.9531073369307344
x = 0.375974
y = 0.0936065
x=0.375974 y=0.0936065
BG:0.8574096110408733
x = 0.450924
y = 0.017396
x=0.450924 y=0.017396
BG:0.768473

NameError: name 'model_evaluation' is not defined

In [71]:
import numpy as np
import scipy
import matplotlib.pyplot as plt 
from numpy.linalg import eig
import matplotlib.animation as animation
from IPython.display import clear_output
import subprocess
import random
from datetime import datetime

def getXY(BG_in):
    # Define the command to run the compiled C++ executable
    executable_path = r'C:\Users\jmcc0\Desktop\FYP\Frank code\getXY.exe'
    run_command = [executable_path]

    # Run the compiled C++ code
    process = subprocess.Popen(run_command, stdin=subprocess.PIPE, stdout=subprocess.PIPE, cwd=r'C:\Users\jmcc0\Desktop\FYP\Frank code')

    # Provide input to the C++ program
    BG = BG_in
    process.stdin.write(str(BG).encode())
    process.stdin.close()

    # Read the output from the C++ program
    output = process.stdout.read().decode().strip()
    x_str, y_str = output.split('\t')
    x = float(x_str.split('=')[1])
    y = float(y_str.split('=')[1])
    # Now you have the x and y values from the C++ code
    print("x =", x)
    print("y =", y)
    
    return x, y

def getGap(layers):
    with open('input.txt', 'w') as f:
        f.write(str(int(len(layers))) + "\n")
        for layer in layers:
            f.write(str(layer.material.name) + " " + str(layer.thickness) + "\n")
    with open('materials.txt', 'w') as f:
        for material in materials:
            f.write(material.name + " " + str(material.affinity) + " " + str(material.band_gap) + " " + str(material.e_eff_mass) + " " + str(material.lh_eff_mass) + " " + str(material.hh_eff_mass) + " " + str(material.refractive) + "\n")
        for material in alloys:
            f.write(material.name + " " + str(material.affinity) + " " + str(material.band_gap) + " " + str(material.e_eff_mass) + " " + str(material.lh_eff_mass) + " " + str(material.hh_eff_mass) + " " + str(material.refractive) + "\n")
    
    executable_path = r'C:\Users\jmcc0\Desktop\FYP\QWI\getGap.exe'
    run_command = [executable_path]
    process = subprocess.Popen(run_command, stdin=subprocess.PIPE, stdout=subprocess.PIPE, cwd=r'C:\Users\jmcc0\Desktop\FYP\QWI')
    
    output = process.stdout.read().decode().strip()
    lines = output.split('\n')

    # Initialize variables to store the values
    energy_e = None
    energy_lh = None
    energy_hh = None

    # Process each line
    for line in lines:
        parts = line.split(':')
        if len(parts) == 2:  # Ensure the line has a label and a value
            label = int(parts[0])
            value = float(parts[1])
            if label == 0:
                energy_e = value
            elif label == 1:
                energy_lh = value
            elif label == 2:
                energy_hh = value
                
    print(max(1240/abs(energy_e-energy_lh), 1240/abs(energy_e-energy_hh)))
    return max(1240/abs(energy_e-energy_lh), 1240/abs(energy_e-energy_hh))

# voigt function details
num_discrete = 2048
func_x = np.zeros(num_discrete)
Gauss_y = np.zeros(num_discrete)
Lorentz_y = np.zeros(num_discrete)
x_0 = num_discrete/2
gamma = 50
sigma = gamma
PLOT_LIMIT=[]

def pad_func_zeros(func):
    func_new = np.zeros(2*len(func))
    j = 0
    for i in range(int(0.25*len(func_new)), int(0.75*len(func_new))):
        func_new[i] = func[j]
        j+=1
    return func_new # twice in length

def pad_func_linear(func):
    func_new = np.zeros(2*len(func))
    j = 0
    del_f = np.abs(func[1]-func[2])
    for i in range(0, int(0.25*len(func_new))):
        func_new[i] = func[0]-0.5*(func[len(func)-1]-func[0]) + i*del_f
    for i in range(int(0.25*len(func_new)), int(0.75*len(func_new))):
        func_new[i] = func[j]
        j+=1
    w=j-1
    j=1
    for i in range(int(0.75*len(func_new)), len(func_new)):
        func_new[i] = func[w] + j*del_f
        j+=1
    return func_new # twice in length

def pad_E(f):
    del_f = np.max(f)/(num_discrete-1)
    func_new = np.zeros(2*len(f))
    j = 0
    for i in range(len(func_new)):
        func_new[i] = del_f*i
    return func_new # twice in length

def convolve(f, g): # PAD ARRAYS BEFORE USE FRO ABSORPTION
    FFT_f = np.fft.fft(f)
    FFT_g = np.fft.fft(g)
    FG = FFT_f * FFT_g
    result = np.fft.ifft(FG)
    return np.real(result)

# plotting
PLOT_LIMIT = []#400,800]
Y_LIMIT = [] # leave blank for auto
ERROR_BARS = False
PLOT_FITTED = False
split_ = ","
LABEL_FONT_SIZE = 13
TICK_FONT_SIZE = 11
LINE_WIDTH = 0.5
MARKER_SIZE = 1
LEGEND = True
label_x = ""
label_y = ""
plot_title = ""
aspect_ratio = [9,9]

colours = [ 'black', 'dimgrey', 'lightslategrey', 'lightsteelblue', 'silver', 'cadetblue', 'darkcyan', 'darkslategray', 'seagreen', 'mediumseagreen', 'darkolivegreen', 'olivedrab', 'olive', 'yellowgreen', 'green', 'springgreen', 'mediumspringgreen', 'turquoise', 'lightseagreen']

def plot_graph(x, y): #create a single plot
    labels = []
    plt.figure()
    plt.rcParams["figure.figsize"] = (aspect_ratio[0],aspect_ratio[1])
    fig, ax = plt.subplots()
    plt.title(plot_title)
    plt.xlabel(label_x, fontsize=LABEL_FONT_SIZE)
    plt.ylabel(label_y,  fontsize=LABEL_FONT_SIZE)
    plt.xticks(fontsize = TICK_FONT_SIZE)
    plt.yticks(fontsize = TICK_FONT_SIZE)
    if(bool(Y_LIMIT) == True):
        plt.ylim(Y_LIMIT)
    if(bool(PLOT_LIMIT) == True):
        plt.xlim(PLOT_LIMIT)
    
    right_side = ax.spines["right"]
    top_side = ax.spines["top"]
    right_side.set_visible(False)
    top_side.set_visible(False)
    plt.plot(x, y, linewidth = LINE_WIDTH, color = 'dimgrey', marker = 's', markersize = MARKER_SIZE, markerfacecolor='dimgrey')
    plt.grid(True, alpha=0.2)
    #labels = np.array(labels)
    #plt.savefig(f'{file}_figure.png', dpi = 1000, bbox_inches='tight')
    plt.show()

def plot_graphs(x, y): #create a single plot
    labels = []
    plt.figure()
    plt.rcParams["figure.figsize"] = (6,6)
    fig, ax = plt.subplots()
    plt.title(plot_title)
    plt.xlabel(label_x, fontsize=LABEL_FONT_SIZE)
    plt.ylabel(label_y,  fontsize=LABEL_FONT_SIZE)
    #ax.xaxis.set_minor_locator(AutoMinorLocator())
    #plt.xticks(fontsize = TICK_FONT_SIZE)
    #plt.yticks(fontsize = TICK_FONT_SIZE)
    if(bool(Y_LIMIT) == True):
        plt.ylim(Y_LIMIT)
    if(bool(PLOT_LIMIT) == True):
        plt.xlim(PLOT_LIMIT)
    
    right_side = ax.spines["right"]
    top_side = ax.spines["top"]
    right_side.set_visible(False)
    top_side.set_visible(False)
    i=0
    for ys in y:
        if i < len(colours):
            plt.plot(x, ys, linewidth = LINE_WIDTH, marker = 's', markersize = MARKER_SIZE, color = colours[i])
        else:
            plt.plot(x, ys, linewidth = LINE_WIDTH, marker = 's', markersize = MARKER_SIZE)
        i+=1
    plt.grid(True, alpha=0.2)
    if(LEGEND==True):
        plt.legend(legend)
    #labels = np.array(labels)
    #plt.savefig(f'{file}_figure.png', dpi = 1000, bbox_inches='tight')
    plt.show()
    
def plot_graphs_distinct(x, y): #create a single plot
    labels = []
    plt.figure()
    plt.rcParams["figure.figsize"] = (6,6)
    fig, ax = plt.subplots()
    plt.title(plot_title)
    plt.xlabel(label_x, fontsize=LABEL_FONT_SIZE)
    plt.ylabel(label_y,  fontsize=LABEL_FONT_SIZE)
    #ax.xaxis.set_minor_locator(AutoMinorLocator())
    #plt.xticks(fontsize = TICK_FONT_SIZE)
    #plt.yticks(fontsize = TICK_FONT_SIZE)
    if(bool(Y_LIMIT) == True):
        plt.ylim(Y_LIMIT)
    if(bool(PLOT_LIMIT) == True):
        plt.xlim(PLOT_LIMIT)
    
    right_side = ax.spines["right"]
    top_side = ax.spines["top"]
    right_side.set_visible(False)
    top_side.set_visible(False)
    i=0
    for ys in y:
        if i < len(colours):
            plt.plot(x[i], ys, linewidth = LINE_WIDTH, marker = 's', markersize = MARKER_SIZE, color = colours[i])
        else:
            plt.plot(x[i], ys, linewidth = LINE_WIDTH, marker = 's', markersize = MARKER_SIZE)
        i+=1
    plt.grid(True, alpha=0.2)
    if(LEGEND==True):
        plt.legend(legend)
    #labels = np.array(labels)
    #plt.savefig(f'{file}_figure.png', dpi = 1000, bbox_inches='tight')
    plt.show()

def make_array(y, number_steps):
    p = np.zeros(number_steps)
    for nr in range(0, number_steps):
        p[nr] = y
    return p

def read_in(file_path):
    x_data_ = []
    y_data_ = []
    try:
        with open(file_path, 'r') as file:
            for line in file:
                # Split the line into x and y values
                x_val, y_val = map(float, line.strip().split())
                x_data_.append(x_val)
                y_data_.append(y_val)
    except FileNotFoundError:
        print("File not found:", file_path)

    # Convert lists to numpy arrays
    x_array_ = np.array(x_data_)
    y_array_ = np.array(y_data_)
    
    #plot_graph(x_array_, y_array_)
    
    return x_array_, y_array_

In [21]:
def write_simulation_parameters(inter_mixing_params, num_electric_fields, max_electric_field):
    with open('simulation_parameters.txt', 'w') as f:
        f.write(f"{inter_mixing_params[0]} {inter_mixing_params[1]}\n")
        f.write(f"{num_electric_fields}\n")
        f.write(f"{max_electric_field}\n")

class Layer:
    def __init__(self, material, thickness): # thickness in [A]
        self.material = material
        self.thickness = thickness

class Material:
    def __init__(self, name, affinity, band_gap, e_eff_mass, lh_eff_mass, hh_eff_mass, refractive):
        self.affinity = affinity
        self.band_gap = band_gap
        self.e_eff_mass = e_eff_mass
        self.lh_eff_mass = lh_eff_mass
        self.hh_eff_mass = hh_eff_mass
        self.name = name
        self.refractive = refractive
        
    def getEffectiveMass(self, p):
        if p == 0:
            return self.e_eff_mass
        if p == 1:
            return self.lh_eff_mass
        if p == 2:
            return self.hh_eff_mass
        
    def getBandgap(self):
        return self.band_gap
    
    def getRefractive(self):
        return self.refractive
    
# Example Materials // BG = Bandgap, EF = Effective Electron Affinity for placing bands [REF: Takuya IEEE Quantum Electronics Vol 30, NO.2]
# Decleration: Material(EF, BG, e_eff_mass, lh_eff_mass, hh_eff_mass, refractive index) 
GaAs = Material("GaAs", 0.111, 1.42, 0.063, 0.082, 0.51, 3.9476)
GaP = Material("GaP", -0.388, 2.74, 0.25, 0.14, 0.67, 3.3798)
InP = Material("InP", 0.0, 1.35, 0.077, 0.14, 0.6, 3.3688)
InAs = Material("InAs", 0.441, 0.354, 0.023, 0.026, 0.41, 3.714)
AlAs = Material("AlAs", -0.4245, 2.95, 0.15, 0.16, 0.79, 2.9940) 
materials = [GaAs, GaP, InP, InAs, AlAs]
# Simulation setup :: InGaAlAs
alloys = []
def BG_InGaAlAs(x, y):
    return 0.36 + 2.093*y + 0.629*x + 0.577*y*y + 0.436*x*x + 1.013*x*y - 2.0*x*y*(1-x-y); # [eV]
def EF_InGaAlAs(x, y): # Effective electron finity for placing conduction bands InGaAlAs
    return 0.5766 - 0.3439*BG_InGaAlAs(x, y) # [eV] 
def effMass_InGaAlAs(x, y, particle):
    return InAs.getEffectiveMass(particle)*(1-x-y) + GaAs.getEffectiveMass(particle)*(x) + AlAs.getEffectiveMass(particle)*(y);
def refractive_InGaAlAs(x, y):
    E_g = BG_InGaAlAs(x,y);
    x=(E_g-0.75)/0.72;
    w = 1240/E_g
    if (x>1.0): x = 1
    A = 9.689 - 1.012*x;
    B = 1.590 - 0.376*x;
    C = 1102.4 - 702.0*x + 330.4*x*x;
    if (C+100 < w):
        X = w*w-C*C
    else: 
        X = (200*w+10000)
    return np.sqrt(A + B*w*w/X);
def InGaAlAs_material(bandgap): # in nm
    bandgap = 1240/bandgap
    x, y = getXY(bandgap)
    print("x="+str(x)+" y="+str(y))
    temp = Material("In{}Ga{}Al{}As".format(1-x-y,x,y), EF_InGaAlAs(x, y), BG_InGaAlAs(x, y), effMass_InGaAlAs(x, y, 0), effMass_InGaAlAs(x, y, 1), effMass_InGaAlAs(x, y, 2), refractive_InGaAlAs(x,y))
    print("BG:" + str(BG_InGaAlAs(x, y)))
    alloys.append(temp)
    return temp

def findAlloys(layers, target):
    layers[0].thickness = 100
    layers[-1].thickness = 100
    sigma = 2500
    sigma0 = 500
    sigma_prev = sigma0
    i = 0
    while i < 20:
        for layer in layers:
            if layer.material.name != 'InP':  # Exclude InP layers from modification
                old_bandgap = 1240/layer.material.band_gap
                new_bandgap = old_bandgap + (target - getGap(layers))  # Adjust each layer's bandgap individually
                layer.material = InGaAlAs_material(new_bandgap)
        with open('input.txt', 'w') as f:
            f.write(str(int(len(layers))) + "\n")
            for layer in layers:
                f.write(str(layer.material.name) + " " + str(layer.thickness) + "\n")

        with open('materials.txt', 'w') as f:
            for material in materials:
                f.write(material.name + " " + str(material.affinity) + " " + str(material.band_gap) + " " + str(material.e_eff_mass) + " " + str(material.lh_eff_mass) + " " + str(material.hh_eff_mass) + " " + str(material.refractive) + "\n")
            for material in alloys:
                f.write(material.name + " " + str(material.affinity) + " " + str(material.band_gap) + " " + str(material.e_eff_mass) + " " + str(material.lh_eff_mass) + " " + str(material.hh_eff_mass) + " " + str(material.refractive) + "\n")

        testGap = getGap(layers)
        temp = sigma
        print(testGap)
        if testGap > target + 1:
            sigma = (sigma0 + sigma) / 2.0
            sigma_prev = temp
        elif testGap < target - 1:
            sigma0 = sigma
            sigma = (sigma_prev + sigma) / 2.0
        else:
            break
        i += 1
    if(i>=20): success = False
    else: success = True
    print("out: " + str(testGap))
    return layers, success

In [49]:
def parse_input_file(input_file):
    layers = []
    with open(input_file, 'r') as f:
        num_layers = int(f.readline().strip())
        for _ in range(num_layers):
            layer_info = f.readline().strip().split()
            material = layer_info[0]
            thickness = float(layer_info[1])
            layers.append((material, thickness))
    return layers

def parse_materials_file(materials_file):
    materials = {}
    with open(materials_file, 'r') as f:
        for line in f:
            material_info = line.strip().split()
            material = material_info[0]
            properties = [float(prop) for prop in material_info[1:]]
            materials[material] = properties
    return materials

def parse_simulation_parameters_file(simulation_params_file):
    with open(simulation_params_file, 'r') as f:
        qwi_target_shift = float(f.readline().split(" ")[1])
        num_electric_fields = int(f.readline().strip())
        max_applied_efield = float(f.readline().strip())
    return qwi_target_shift, num_electric_fields, max_applied_efield

def write_data_to_file(filename, data_dict):
    with open(filename, 'w') as f:
        for key, value in data_dict.items():
            f.write(f"{key}:\n")
            if isinstance(value, list):
                for item in value:
                    f.write(f"{item}\n")
            elif isinstance(value, dict):
                for subkey, subvalue in value.items():
                    f.write(f"{subkey}: {subvalue}\n")
            else:
                f.write(f"{value}\n")

def read_data_from_file(filename):
    data_dict = {}
    with open(filename, 'r') as f:
        lines = f.readlines()
        i = 0
        while i < len(lines):
            key = lines[i].strip(":\n")
            i += 1
            if key in ["Layers", "Materials"]:
                data_dict[key] = {}
                while i < len(lines) and lines[i] != "\n":
                    line = lines[i].strip("\n")
                    if line and ":" in line:
                        subkey, subvalue = line.split(": ", 1)  # Limit split to 1 occurrence
                        data_dict[key][subkey] = subvalue
                    i += 1
            else:
                if i < len(lines):
                    value = lines[i].strip("\n")
                    data_dict[key] = value
                    i += 1
    return data_dict

In [58]:
max_electric_field = 10
def objective_function_model():
    input_file = "input.txt"
    materials_file = "materials.txt"
    simulation_params_file = "simulation_parameters.txt"
    layers = parse_input_file(input_file)
    materials_out = parse_materials_file(materials_file)
    FOM_data = [0, 0, 0, 0]
    qwi_target_shift, num_electric_fields, efield = parse_simulation_parameters_file(simulation_params_file)
    print("Layers:", layers)
    # print("Materials:", materials_out)
    print("QWI Target Shift:", qwi_target_shift)
    print("Number of Electric Fields:", num_electric_fields)
    print("Max Applied Electric Field:", max_electric_field)
    print("FOM:", FOM_data)

    data_dict = {
        "Layers": layers,
        "Materials": materials_out,
        "QWI Target Shift": qwi_target_shift,
        "Number of Electric Fields": num_electric_fields,
        "Max Applied Electric Field": max_electric_field,
        "FOM": FOM_data
    }
    write_data_to_file("optimised.txt", data_dict)
    file_path = r"optimised.txt"
    # Read the data from the file
    materials, layers, qwi_target_shift, max_applied_electric_field, fom_elements = read_data(file_path)

    # Extract material names from the 'Layers' data in order of appearance
    material_names_ordered = [layer[0] for layer in layers]

    # Flatten the materials dictionary in the order of appearance
    material_features = []
    for material_key in material_names_ordered:
        if material_key in materials:
            material_features.extend(materials[material_key])

    # Flatten the layers list
    layer_thicknesses = [thickness for _, thickness in layers]

    # Combine all features into a single array
    all_features = np.concatenate((material_features, layer_thicknesses, [qwi_target_shift], [max_applied_electric_field]))

    input_tensor = torch.tensor(all_features, dtype=torch.float32)
    print(input_tensor)
    # Pass input data through the model
    with torch.no_grad():
        model.eval()  # Set the model to evaluation mode
        output_tensor = model(input_tensor)

    # Get the predicted values
    predicted_values = output_tensor.numpy()

    print("Predicted values:", predicted_values)
    return predicted_values[2]
    
objective_function_model()
    

Layers: [('In0.5308573999999999Ga0.402031Al0.0671116As', 100.0), ('In0.528136Ga0.2402Al0.231664As', 88.9827589220059), ('In0.5270600000000001Ga0.176191Al0.296749As', 58.744346491572486), ('In0.529802Ga0.339226Al0.130972As', 62.907533859634434), ('In0.5308573999999999Ga0.402031Al0.0671116As', 100.0)]
QWI Target Shift: 0.0
Number of Electric Fields: 10
Max Applied Electric Field: 10
FOM: [0, 0, 0, 0]
tensor([2.9285e-01, 8.2510e-01, 4.7604e-02, 5.7507e-02, 4.7571e-01, 3.5386e+00,
        2.1562e-01, 1.0497e+00, 6.2029e-02, 7.0494e-02, 5.2205e-01, 3.5183e+00,
        1.7970e-01, 1.1541e+00, 6.7735e-02, 7.5631e-02, 5.4038e-01, 3.5131e+00,
        2.6520e-01, 9.0550e-01, 5.3202e-02, 6.2547e-02, 4.9369e-01, 3.5320e+00,
        2.9285e-01, 8.2510e-01, 4.7604e-02, 5.7507e-02, 4.7571e-01, 3.5386e+00,
        1.0000e+02, 8.8983e+01, 5.8744e+01, 6.2908e+01, 1.0000e+02, 0.0000e+00,
        1.0000e+01])
Predicted values: [ 0.32988483 -0.19017376  2.4211264   0.01276863]


2.4211264

In [93]:
import numpy as np
from skopt import gp_minimize
from skopt.space import Real
from skopt.utils import use_named_args

max_electric_field = 10
def objective_function_model():
    input_file = "input.txt"
    materials_file = "materials.txt"
    simulation_params_file = "simulation_parameters.txt"
    layers = parse_input_file(input_file)
    materials_out = parse_materials_file(materials_file)
    FOM_data = [0, 0, 0, 0]
    qwi_target_shift, num_electric_fields, efield = parse_simulation_parameters_file(simulation_params_file)
    print("Layers:", layers)
    # print("Materials:", materials_out)
    print("QWI Target Shift:", qwi_target_shift)
    print("Number of Electric Fields:", num_electric_fields)
    print("Max Applied Electric Field:", max_electric_field)
    print("FOM:", FOM_data)

    data_dict = {
        "Layers": layers,
        "Materials": materials_out,
        "QWI Target Shift": qwi_target_shift,
        "Number of Electric Fields": num_electric_fields,
        "Max Applied Electric Field": max_electric_field,
        "FOM": FOM_data
    }
    write_data_to_file("optimised.txt", data_dict)
    file_path = r"optimised.txt"
    # Read the data from the file
    materials, layers, qwi_target_shift, max_applied_electric_field, fom_elements = read_data(file_path)

    # Extract material names from the 'Layers' data in order of appearance
    material_names_ordered = [layer[0] for layer in layers]

    # Flatten the materials dictionary in the order of appearance
    material_features = []
    for material_key in material_names_ordered:
        if material_key in materials:
            material_features.extend(materials[material_key])

    # Flatten the layers list
    layer_thicknesses = [thickness for _, thickness in layers]

    # Combine all features into a single array
    all_features = np.concatenate((material_features, layer_thicknesses, [qwi_target_shift], [max_applied_electric_field]))

    input_tensor = torch.tensor(all_features, dtype=torch.float32)
    print(input_tensor)
    # Pass input data through the model
    with torch.no_grad():
        model.eval()  # Set the model to evaluation mode
        output_tensor = model(input_tensor)

    # Get the predicted values
    predicted_values = output_tensor.numpy()

    print("Predicted values:", predicted_values)
    return predicted_values[2]
    
objective_function_model()
    

# Define the search space for thicknesses and bandgaps
space_thickness = [Real(27, 60, name=f'thickness_{i}') for i in range(3)]  # Thickness for layers 1, 2, 3
space_bandgap = [Real(1000, 1650, name=f'bandgap_{i}') for i in range(4)]  # Bandgap for all layers

count = 0
successes = 0

# Additional constraint: The first and last layers should have the same bandgap
def objective_function(thicknesses, bandgaps):
    # Set the thickness for the layers
    thicknesses = [100] + thicknesses + [100]
    global count
    global successes
    count+=1
    # Set the bandgap for the layers
    # Ensure the last bandgap is equal to the first
    bandgaps_reordered = [bandgaps[0]] + bandgaps[1:] + [bandgaps[0]]
    
    # Check if the bandgap of the entire heterostructure is within the tolerance
    bandgap_tolerance = 1.5
    target_bandgap = 1500
    
    global alloys
    alloys=[]
    # Create the heterostructure
    layers = [Layer(InGaAlAs_material(bandgaps_reordered[i]), thicknesses[i]) for i in range(5)]  # Use bandgaps from optimization space
    gapp = getGap(layers)
    # Check the bandgap of the entire heterostructure
    if abs(gapp - target_bandgap) <= bandgap_tolerance:
        # Evaluate the figure of merit using the model
        fom = objective_function_model()
        clear_output()
        print("true")
        print(count)
        print("FOM: " + str(fom))
        print("GAP: " + str(getGap(layers)))
        successes+=1
        return -fom  # Return figure of merit for maximization
    else:
        clear_output()
        print("false")
        print(count)
        penalty = abs(gapp - target_bandgap)*2
        print("penalty: " + str(penalty))
        # Return a large positive value for designs with bandgap not within the tolerance
        return penalty  # penalty for designs not meeting the constraint

# Define the objective function using the trained model
@use_named_args(space_thickness + space_bandgap)
def objective(**params):
    thicknesses = [params[f'thickness_{i}'] for i in range(3)]  # Only consider thicknesses for layers 1, 2, 3
    bandgaps = [params[f'bandgap_{i}'] for i in range(4)]  # Consider bandgaps for all layers
    return objective_function(thicknesses, bandgaps)

# InP_layer2 = Layer(InGaAlAs_material(1239.85), 100)
# layer1 = Layer(InGaAlAs_material(1650.85), 49)
# layer2 = Layer(InGaAlAs_material(1400.85), 60)
# layer3 = Layer(InGaAlAs_material(1000.00), 40)
# layer4 = Layer(InGaAlAs_material(1650.85), 30)

# layers = [InP_layer2, layer4, layer2, layer1, InP_layer2]

# Initial guess for thicknesses and bandgaps
initial_thicknesses = [30, 30, 60]  # Example initial guess for thicknesses
initial_bandgaps = [1194.766898954708, 1624.720691840501, 1369.6999642066814, 1617.95942947708]  # Example initial guess for bandgaps

# Concatenate initial guesses
initial_guess = initial_thicknesses + initial_bandgaps

# Optimize the heterostructure composition starting from the initial guess
result = gp_minimize(objective, space_thickness + space_bandgap, x0=initial_guess, n_calls=300)

# Get the optimized thicknesses and bandgaps
optimized_thicknesses = result.x[:3]  # First 3 elements correspond to thicknesses
optimized_bandgaps = result.x[3:]    # Last 4 elements correspond to bandgaps

print("Optimized thicknesses:", optimized_thicknesses)
print("Optimized bandgaps:", optimized_bandgaps)


false
200
penalty: 8.887704236886293
Optimized thicknesses: [27.0, 58.15505279291747, 60.0]
Optimized bandgaps: [1184.3736555850817, 1523.37091950054, 1412.1710271335708, 1625.1830535398785]


In [85]:
import numpy as np
from skopt import gp_minimize
from skopt.space import Real
from skopt.utils import use_named_args
from deap import base, creator, tools

# Define the evaluation function (objective function)
def evaluate(individual):
    thicknesses = individual[:3]
    bandgaps = individual[3:]
    return objective_function(thicknesses, bandgaps),

# Define the individual creation function
def create_individual():
    thicknesses = [random.uniform(30, 100) for _ in range(3)]
    bandgaps = [random.uniform(1000, 1660) for _ in range(4)]
    return thicknesses + bandgaps

# Register necessary DEAP components
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)
toolbox = base.Toolbox()
toolbox.register("individual", tools.initIterate, creator.Individual, create_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", evaluate)
toolbox.register("mate", tools.cxBlend, alpha=0.5)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.1)
toolbox.register("select", tools.selTournament, tournsize=3)

# Define the objective function
def objective_function(thicknesses, bandgaps):
    # Set the thickness for the layers
    thicknesses = [100] + thicknesses + [100]
    global count
    count+=1
    # Set the bandgap for the layers
    # Ensure the last bandgap is equal to the first
    bandgaps_reordered = [bandgaps[0]] + bandgaps[1:] + [bandgaps[0]]
    
    # Check if the bandgap of the entire heterostructure is within the tolerance
    bandgap_tolerance = 1.5
    target_bandgap = 1500
    
    global alloys
    alloys=[]
    # Create the heterostructure
    layers = [Layer(InGaAlAs_material(bandgaps_reordered[i]), thicknesses[i]) for i in range(5)]  # Use bandgaps from optimization space
    
    # Check the bandgap of the entire heterostructure
    if abs(getGap(layers) - target_bandgap) <= bandgap_tolerance:
        # Evaluate the figure of merit using the model
        fom = objective_function_model()
        clear_output()
        print("true")
        print(count)
        print("FOM: " + str(fom))
        print("GAP: " + str(getGap(layers)))
        return -fom  # Return figure of merit for maximization
    else:
        clear_output()
        print("false")
        print(count)
        # Return a large positive value for designs with bandgap not within the tolerance
        return 1  # Large penalty for designs not meeting the constraint

# Remaining parts of your code

# Run the genetic algorithm
def genetic_algorithm():
    population_size = 50
    num_generations = 50
    population = toolbox.population(n=population_size)
    
    for gen in range(num_generations):
        offspring = toolbox.select(population, len(population))
        offspring = list(map(toolbox.clone, offspring))

        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < 0.5:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values
        
        for mutant in offspring:
            if random.random() < 0.2:
                toolbox.mutate(mutant)
                del mutant.fitness.values
        
        invalid_individuals = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_individuals)
        for ind, fit in zip(invalid_individuals, fitnesses):
            ind.fitness.values = fit
        
        population[:] = offspring
    
    return tools.selBest(population, k=1)[0]

best_individual = genetic_algorithm()
optimized_thicknesses = best_individual[:3]
optimized_bandgaps = best_individual[3:]

print("Optimized thicknesses:", optimized_thicknesses)
print("Optimized bandgaps:", optimized_bandgaps)


false
286
x = 0.214499
y = 0.257797
x=0.214499 y=0.257797
BG:1.090551242535432
x = 0.276844
y = 0.194403
x=0.276844 y=0.194403
BG:0.9938475345347901
x = 0.368528
y = 0.101178
x=0.368528 y=0.101178
BG:0.8669164162977655
x = 0.321674
y = 0.14882
x=0.321674 y=0.14882
BG:0.9295043215735599
x = 0.214499
y = 0.257797
x=0.214499 y=0.257797
BG:1.090551242535432


KeyboardInterrupt: 